# ORGANIZADOR DE CARPETAS Y DOCS

Rutas originales de archivos

'/Users/miguelangelmayordomogragera/Documents/TRABAJO/CIBERNOS/ARCHIVOS MARZO 2024/ORGANIZADOR CARPETAS Y DOCS/ORGANIZADOR_CARPETAS_DOCS.py'
'/Users/miguelangelmayordomogragera/Documents/TRABAJO/CIBERNOS/ARCHIVOS MARZO 2024/MACRO EXCEL/ORGANIZADOR_CARPETAS v2.xlsm'


## ARCHIVO PYTHON

In [ ]:
# Importación de librerías principales
import pandas as pd
import os
import re
import shutil

### Carga de Datos

In [ ]:
# Datos a cargar
df_datos=pd.read_excel('C:/Users/ES45877372P/OneDrive - Enel Spa/ARCHIVOS_TRABAJO/ORGANIZADOR CARPETAS Y DOCS/OneDrive_1_19-4-2023/01_MT_eDRD/eDRD_MT_FUERZA MAYOR_2022.xlsx',sheet_name='Muestra MT FM',dtype=str)

# Ruta de la carpeta principal
ruta_principal = r'C:/Users/ES45877372P/OneDrive - Enel Spa/ARCHIVOS_TRABAJO/ORGANIZADOR CARPETAS Y DOCS/MT_FM'

# Ruta de la carpeta con los documentos a mover
ruta_documentos = os.path.join('C:/Users/ES45877372P/Desktop/DOCS MT FM')

### Movimiento de archivos

In [ ]:
# Crea la carpeta principal si no existe
if not os.path.exists(ruta_principal):
    os.mkdir(ruta_principal)

# Agrupa los datos por identificador y referencia
grupos = df_datos.groupby(['ZONA', 'Referencia'])

# Itera sobre los grupos y crea las carpetas correspondientes
for (zona, referencia), _ in grupos:
    # Crea la carpeta para el identificador si no existe
    carpeta_identificador = os.path.join(ruta_principal, zona)
    if not os.path.exists(carpeta_identificador):
        os.mkdir(carpeta_identificador)
    
    # Crea la carpeta para la referencia si no existe
    carpeta_referencia = os.path.join(carpeta_identificador, referencia)
    if not os.path.exists(carpeta_referencia):
        os.mkdir(carpeta_referencia)

In [ ]:
# Itera sobre los grupos y mueve los archivos correspondientes
for (zona, referencia), grupo in grupos:
    # Ruta de la carpeta correspondiente a la referencia
    carpeta_referencia = os.path.join(ruta_principal, zona, referencia)
    
    # Itera sobre los archivos en la carpeta de documentos
    for archivo in os.listdir(ruta_documentos):
        # Busca el número de referencia en el nombre del archivo
        if re.search(referencia, archivo):
            # Ruta completa del archivo a mover
            ruta_origen = os.path.join(ruta_documentos, archivo)
            
            # Ruta completa de destino del archivo
            ruta_destino = os.path.join(carpeta_referencia, archivo)
            
            # Mueve el archivo a la carpeta correspondiente
            shutil.move(ruta_origen, ruta_destino)

In [ ]:
#Carpetas vacias
# Lista de carpetas vacías
carpetas_vacias = []

# Itera sobre los grupos y verifica si las carpetas están vacías
for (zona, referencia), _ in grupos:
    # Ruta de la carpeta correspondiente a la referencia
    carpeta_referencia = os.path.join(ruta_principal, zona, referencia)
    
    # Verifica si la carpeta está vacía
    if not os.listdir(carpeta_referencia):
        carpetas_vacias.append(os.path.basename(carpeta_referencia))

# Imprime la lista de carpetas vacías
print(carpetas_vacias)

## CÓDIGO VBA
Este código es para una macro de Excel, cuyas instrucciones son las que se muestran a continuación:

![Captura de pantalla](./AUTOMATISMOS/ORGANIZADOR_CARPETAS_DOCS/img/c1.png)

![Captura de pantalla](./AUTOMATISMOS/ORGANIZADOR_CARPETAS_DOCS/img/c2.png)

El aspecto de la hoja principal es este

![Captura de pantalla](./img/c3.png)

Esto solo se verá en GitHub, no afecta localmente

![Captura](https://raw.githubusercontent.com/mamg97/AI4G_mamg/main/AUTOMATISMOS/ORGANIZADOR_CARPETAS_DOCS/img/c3.png)




![Screenshot of a comment on a GitHub issue showing an image, added in the Markdown, of an Octocat smiling and raising a tentacle.](https://myoctocat.com/assets/images/base-octocat.svg)

In [ ]:
# Aquí puedes escribir tus conclusiones o próximos pasos
print("Análisis finalizado.")